In [2]:
from warnings import simplefilter, filterwarnings
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)

simplefilter(action='ignore', category=FutureWarning)
filterwarnings('ignore')
#https://docs.anaconda.com/ae-notebooks/user-guide/adv-tasks/work-with-environments/#:~:text=To%20use%20your%20new%20environment,you%20just%20created%20is%20displayed.

ModuleNotFoundError: No module named 'sklearn.utils.testing'

# Menu
<a name="navegacao"></a>

## 1) [Data cleaning and feature engineering](#cleaning)
- 1.1 [Encoding de Conglomerado e Natureza Jurídica](#encoding)
- 1.2 [Descrição dos dados](#descricao)
- 1.3 [Feature engineering](#engineering)
- 1.4 [Train/Test partition](#partition)

    
## 2) [Random Search for Hyperparameters](#randomsearch)
- 2.1 [Extreme Learning Machine](#elm)
- 2.2 [Suport Vector Machine](#svm)
- 2.3 [Stochastic Gradient Descent](#sgd)
- 2.4 [Multi-Task Elastic-Net](#elastic)
- 2.5 [Least Angle Regression Model (LAR)](#LAR)
- 2.6 [Ridge Regression](#ridge)
- 2.7 [ElasticNet Regression](#ElasNet)
- 2.8 [Bayesian Ridge](#bayesian)
- 2.9 [Lasso Regression](#lasso)
- 2.10 [Multilayer Perceptron Regression](#mlp)
- 2.11 [Nearest Neighbors Regression](#nnr)
- 2.12 [Resumo dos resultados](#resumo_rs)


## 3) [Grid Serach for Hyperparameters](#gridsearch)
- 3.1 [Extreme Learning Machine](#elm2)
- 3.2 [Suport Vector Machine](#svm2)
- 3.3 [Stochastic Gradient Descent](#sgd2)
- 3.4 [Multi-Task Elastic-Net](#elastic2)
- 3.5 [Least Angle Regression Model (LAR)](#LAR2)
- 3.6 [Ridge Regression](#ridge2)
- 3.7 [ElasticNet Regression](#ElasNet2)
- 3.8 [Bayesian Ridge](#bayesian2)
- 3.9 [Lasso Regression](#lasso2)
- 3.10 [Multilayer Perceptron Regression](#mlp2)
- 3.11 [Nearest Neighbors Regression](#nnr2)
- 3.12 [Resumo dos resultados](#resumo_rs2)

In [3]:
from time import time
import requests
import pandas as pd
import numpy as np 
import random 
import os
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [6]:
%%time

#spread = pd.read_sas('dados bndes/dadospreadpy.sas7bdat' , encoding='utf-8')
spread = pd.read_sas('dados bndes/dadospreadpy.sas7bdat' , encoding='latin-1')
#https://www.analyticsvidhya.com/blog/2018/03/introduction-regression-splines-python-codes/

Wall time: 1.31 s


### Veja tutorial sobre lags em um painel de dados:

- https://towardsdatascience.com/timeseries-data-munging-lagging-variables-that-are-distributed-across-multiple-groups-86e0a038460c

### Veja tutorial sobre Bayesian Hyperparameter Tunning for ML:

- https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f

<img style="float: left;" src="https://miro.medium.com/max/500/1*yVGF2yzXyJetXgSF3IARQw.jpeg" width="30%"> 

In [7]:
spread.columns

Index(['CNPJ', 'EMPRESA', 'ANO', 'MES', 'ValorOperacao', 'SpreadBNDES',
       'SpreadInstituicaoFinanceira', 'CNPJAgente', 'Juros', 'Prazo_Amortiza',
       'prazo_carencia', 'TRIM', 'POTOT_FIRMA', 'IDADE_FIRMA', 'MSAL_FIRMA',
       'PO3G_FIRMA', 'ROT_FIRMA', 'ENG_FIRMA', 'SHARE_FIRMA', 'NATJUR', 'TCB',
       'Conglomerado', 'Ativo_Total', 'Carteira_Credito', 'Passivo_Circulante',
       'Captacoes', 'Patrimonio_Liquido', 'Lucro_Liquido', 'Numero_Agencias',
       'Numero_Postos'],
      dtype='object')

<a name="cleaning"></a>


# 1) Data Cleaning and Feature Engineering

- ir para [Menu Principal](#navegacao)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Train/Test partition](#partition)


<a name="encoding"></a>

## 1.1) Encoding de Conglomerado e Natureza Jurídica


In [8]:
%%time

spread.fillna(0,inplace=True)

Wall time: 55.3 ms


In [9]:
%%time

enc_congl = spread[['CNPJ','Conglomerado']].groupby(['Conglomerado']).count().reset_index().rename(columns={"CNPJ":"banco_enc"})
enc_congl.head()

Wall time: 18 ms


,Conglomerado,banco_enc
0,0,23462
1,ABC-BRASIL,289
2,ALFA,435
3,BANCOOB,389
4,BANESTES,34


In [10]:
%%time
spread = pd.merge( spread, enc_congl, left_on=['Conglomerado'],right_on=['Conglomerado'], how='inner')

Wall time: 75 ms


In [11]:
%%time

enc_natjur = spread[['CNPJ','NATJUR']].groupby(['NATJUR']).count().reset_index().rename(columns={"CNPJ":"firma_enc"})
enc_natjur.head()
#NATJUR

Wall time: 61.8 ms


,NATJUR,firma_enc
0,1015,6
1,1031,65
2,1112,8
3,1244,2
4,2011,28


In [12]:
%%time
spread = pd.merge( spread, enc_natjur, left_on=['NATJUR'],right_on=['NATJUR'], how='inner')

Wall time: 95.9 ms


In [13]:
spread.loc[ (spread['TCB']=='b1'), 'seg_b1'] = 1
spread.loc[~(spread['TCB']=='b1'), 'seg_b1'] = 0

spread.loc[ (spread['TCB']=='b2'), 'seg_b2'] = 1
spread.loc[~(spread['TCB']=='b2'), 'seg_b2'] = 0

spread.loc[ (spread['TCB']=='b3C'), 'seg_b3'] = 1
spread.loc[~(spread['TCB']=='b3C'), 'seg_b3'] = 0

spread[['CNPJ','EMPRESA','seg_b3']].groupby(['seg_b3']).count()

,CNPJ,EMPRESA
seg_b3,,
0.0,58868,58868
1.0,229,229


In [14]:
spread[['CNPJ','EMPRESA','seg_b3']].groupby(['seg_b3']).count()

,CNPJ,EMPRESA
seg_b3,,
0.0,58868,58868
1.0,229,229


In [15]:

spread[['CNPJ','EMPRESA','TCB']].groupby(['TCB']).count()

,CNPJ,EMPRESA
TCB,,
b1,34647,34647
b2,22218,22218
b3C,229,229
b4,1447,1447
n1,556,556


<a name="engineering"></a>

## 1.2) Feature engineering
- ir para [Menu Principal](#navegacao)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Train/Test partition](#partition)


In [16]:
%%time
spread['target'] = spread['SpreadInstituicaoFinanceira'] 

# dados da operação
spread['oper_valor']                   = np.log(spread['ValorOperacao']) 
spread['oper_spreadbndes']             = spread['SpreadBNDES'] 
spread['oper_juros']                   = spread['Juros'] 
spread['oper_amortizacao']             = np.log(spread['Prazo_Amortiza']) 
spread['oper_carencia']                = np.log(spread['prazo_carencia']+1) 
spread['oper_valor_spreadbndes']       = np.log(spread['ValorOperacao']) - spread['SpreadBNDES']
spread['oper_valor_juros']             = np.log(spread['ValorOperacao']) - spread['Juros']
spread['oper_valor_amortizacao']       = np.log(spread['ValorOperacao']) - spread['Prazo_Amortiza']
spread['oper_valor_carencia']          = np.log(spread['ValorOperacao']) - spread['prazo_carencia'] 
spread['oper_spreadbndes_juros']       = spread['SpreadBNDES']           - spread['Juros'] 
spread['oper_spreadbndes_amortizacao'] = spread['SpreadBNDES']           - spread['Prazo_Amortiza']
spread['oper_spreadbndes_carencia']    = spread['SpreadBNDES']           - spread['prazo_carencia']
spread['oper_juros_amortizacao']       = spread['Juros']                 - spread['Prazo_Amortiza']
spread['oper_juros_carencia']          = spread['Juros']                 - spread['prazo_carencia']
spread['oper_amortizacao_carencia']    = spread['Prazo_Amortiza']        - spread['prazo_carencia']

Wall time: 46.7 ms


In [17]:
%%time
# dados da firma
spread['firma_potot']  = np.log(spread['POTOT_FIRMA']+1) 
spread['firma_idade']  = np.log(spread['IDADE_FIRMA']+1) 
spread['firma_massa']  = np.log(spread['MSAL_FIRMA']+1) 
spread['firma_po3gr']  = np.log(spread['PO3G_FIRMA']+1) 
spread['firma_rotat']  = np.log(spread['ROT_FIRMA']+1) 
spread['firma_engen']  = np.log(spread['ENG_FIRMA']+1) 
spread['firma_share']  = np.log(spread['SHARE_FIRMA']+1) 


spread['firma_potot_idade'] = np.log(spread['POTOT_FIRMA']+1) / np.log(spread['IDADE_FIRMA']+2) 
spread['firma_potot_massa'] = np.log(spread['POTOT_FIRMA']+1) / np.log(spread['MSAL_FIRMA']+2) 
spread['firma_potot_po3gr'] = np.log(spread['POTOT_FIRMA']+1) / np.log(spread['PO3G_FIRMA']+2) 
#spread['firma_potot_rotat'] = np.log(spread['POTOT_FIRMA']+1) / np.log(spread['ROT_FIRMA']+2) 
#spread['firma_potot_engen'] = np.log(spread['POTOT_FIRMA']+1) / np.log(spread['ENG_FIRMA']+2) 
#spread['firma_potot_share'] = np.log(spread['POTOT_FIRMA']+1) / np.log(spread['SHARE_FIRMA']+2) 
spread['firma_idade_massa'] = np.log(spread['IDADE_FIRMA']+1) / np.log(spread['MSAL_FIRMA']+2) 
spread['firma_idade_po3gr'] = np.log(spread['IDADE_FIRMA']+1) / np.log(spread['PO3G_FIRMA']+2) 
#spread['firma_idade_rotat'] = np.log(spread['IDADE_FIRMA']+1) / np.log(spread['ROT_FIRMA']+2) 
spread['firma_idade_engen'] = np.log(spread['IDADE_FIRMA']+1) / np.log(spread['ENG_FIRMA']+2)
#spread['firma_idade_share'] = np.log(spread['IDADE_FIRMA']+1) / np.log(spread['SHARE_FIRMA']+2)
spread['firma_massa_po3gr'] = np.log(spread['MSAL_FIRMA']+1)  / np.log(spread['PO3G_FIRMA']+2) 
#spread['firma_massa_rotat'] = np.log(spread['MSAL_FIRMA']+1)  / np.log(spread['ROT_FIRMA']+2) 
spread['firma_massa_engen'] = np.log(spread['MSAL_FIRMA']+1)  / np.log(spread['ENG_FIRMA']+2)
#spread['firma_massa_share'] = np.log(spread['MSAL_FIRMA']+1)  / np.log(spread['SHARE_FIRMA']+2)
#spread['firma_po3gr_rotat'] = np.log(spread['PO3G_FIRMA']+1)  / np.log(spread['ROT_FIRMA']+2) 
spread['firma_po3gr_engen'] = np.log(spread['PO3G_FIRMA']+1)  / np.log(spread['ENG_FIRMA']+2)
#spread['firma_po3gr_share'] = np.log(spread['PO3G_FIRMA']+1)  / np.log(spread['SHARE_FIRMA']+2)
spread['firma_rotat_engen'] = np.log(spread['ROT_FIRMA']+1)   / np.log(spread['ENG_FIRMA']+2)
spread['firma_rotat_share'] = np.log(spread['ROT_FIRMA']+1)   - np.log(spread['SHARE_FIRMA']+2)
spread['firma_engen_share'] = np.log(spread['ENG_FIRMA']+1)   - np.log(spread['SHARE_FIRMA']+2)

Wall time: 32.9 ms


In [18]:
# dados da instituição financeira
spread['banco_ativo']        = np.log(spread['Ativo_Total']) 
spread['banco_carteira']     = np.log(spread['Carteira_Credito']) 
spread['banco_passivo']      = np.log(spread['Passivo_Circulante']) 
spread['banco_captacoes']    = np.log(spread['Captacoes']+1) 
spread['banco_patrimonio']   = np.log(spread['Patrimonio_Liquido']+1) 
spread['banco_lucroliquido'] = np.log(spread['Lucro_Liquido']+1) 
spread['banco_numagencias']  = np.log(spread['Numero_Agencias']+1) 
spread['banco_numeropostos'] = np.log(spread['Numero_Postos']+1) 

spread['banco_ativo_carteira']            =np.log(spread['Ativo_Total'])        /np.log(spread['Carteira_Credito'])   
spread['banco_ativo_passivo']             =np.log(spread['Ativo_Total'])        /np.log(spread['Passivo_Circulante']) 
spread['banco_ativo_captacoes']           =np.log(spread['Ativo_Total'])        /np.log(spread['Captacoes']+1)        
spread['banco_ativo_patrimonio']          =np.log(spread['Ativo_Total'])        /np.log(spread['Patrimonio_Liquido']) 
#spread['banco_ativo_lucroliquido']        =np.log(spread['Ativo_Total'])        /np.log(spread['Lucro_Liquido']+1)    
#spread['banco_ativo_numagencias']         =np.log(spread['Ativo_Total'])        /np.log(spread['Numero_Agencias']+1)  
#spread['banco_ativo_numeropostos']        =np.log(spread['Ativo_Total'])        /np.log(spread['Numero_Postos']+1)    
spread['banco_carteira_passivo']          =np.log(spread['Carteira_Credito'])   /np.log(spread['Passivo_Circulante']) 
spread['banco_carteira_captacoes']        =np.log(spread['Carteira_Credito'])   /np.log(spread['Captacoes']+1) 
spread['banco_carteira_patrimonio']       =np.log(spread['Carteira_Credito'])   /np.log(spread['Patrimonio_Liquido']) 
#spread['banco_carteira_lucroliquido']     =np.log(spread['Carteira_Credito'])   /np.log(spread['Lucro_Liquido']+1) 
#spread['banco_carteira_numagencias']      =np.log(spread['Carteira_Credito'])   /np.log(spread['Numero_Agencias']+1) 
#spread['banco_carteira_numeropostos']     =np.log(spread['Carteira_Credito'])   /np.log(spread['Numero_Postos']+1) 
spread['banco_passivo_captacoes']         =np.log(spread['Passivo_Circulante']) /np.log(spread['Captacoes']+1) 
spread['banco_passivo_patrimonio']        =np.log(spread['Passivo_Circulante']) /np.log(spread['Patrimonio_Liquido']) 
#spread['banco_passivo_lucroliquido']      =np.log(spread['Passivo_Circulante']) /np.log(spread['Lucro_Liquido']+1) 
#spread['banco_passivo_numagencias']       =np.log(spread['Passivo_Circulante']) /np.log(spread['Numero_Agencias']+1) 
#spread['banco_passivo_numeropostos']      =np.log(spread['Passivo_Circulante']) /np.log(spread['Numero_Postos']+1) 
spread['banco_captacoes_patrimonio']      =np.log(spread['Captacoes']+1)        /np.log(spread['Patrimonio_Liquido']+1) 
#spread['banco_captacoes_lucroliquido']    =np.log(spread['Captacoes']+1)        /np.log(spread['Lucro_Liquido']+1) 
#spread['banco_captacoes_numagencias']     =np.log(spread['Captacoes']+1)        /np.log(spread['Numero_Agencias']+1) 
#spread['banco_captacoes_numeropostos']    =np.log(spread['Captacoes']+1)        /np.log(spread['Numero_Postos']+1) 
#spread['banco_patrimonio_lucroliquido']   =np.log(spread['Patrimonio_Liquido']) /np.log(spread['Lucro_Liquido']+1) 
#spread['banco_patrimonio_numagencias']    =np.log(spread['Patrimonio_Liquido']) /np.log(spread['Numero_Agencias']+1) 
#spread['banco_patrimonio_numeropostos']   =np.log(spread['Patrimonio_Liquido']) /np.log(spread['Numero_Postos']+1) 
#spread['banco_lucroliquido_numagencias']  =np.log(spread['Lucro_Liquido'])      /np.log(spread['Numero_Agencias']+1) 
#spread['banco_lucroliquido_numeropostos'] =np.log(spread['Lucro_Liquido'])      /np.log(spread['Numero_Postos']+1) 
#spread['banco_numagencias_numeropostos']  =np.log(spread['Numero_Agencias']+1)  /np.log(spread['Numero_Postos']+1) 


In [19]:
%%time

spread.fillna(0,inplace=True)

#spread.replace([np.inf, -np.inf],0, inplace=True)

Wall time: 72.4 ms


<a name="descricao"></a>

## 1.3) Descrição dos dados
- ir para [Menu Principal](#navegacao)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Train/Test partition](#partition)

In [20]:
%%time

spread.filter(regex='firma', axis=1).describe().transpose()

#spread[['POTOT_FIRMA','IDADE_FIRMA','MSAL_FIRMA','PO3G_FIRMA','ROT_FIRMA','ENG_FIRMA','SHARE_FIRMA',
#        'firma_potot','firma_idade','firma_massa','firma_po3gr','firma_rotat','firma_engen',
#        'firma_share']].describe().transpose()


Wall time: 114 ms


,count,mean,std,min,25%,50%,75%,max
firma_enc,59097.0,33605.668257,17273.775945,1.000000e+00,6909.000000,43792.000000,43792.000000,43792.000000
firma_potot,59097.0,3.373903,1.827505,4.987542e-03,1.951040,3.063391,4.617672,9.556813
firma_idade,59097.0,2.703713,0.840707,0.000000e+00,2.149434,2.761907,3.311273,4.390367
firma_massa,59097.0,10.781991,2.256507,0.000000e+00,9.172427,10.538891,12.404739,18.422188
firma_po3gr,59097.0,1.426054,1.609216,0.000000e+00,0.000000,0.747635,2.475277,9.104200
firma_rotat,59097.0,0.218042,0.103674,0.000000e+00,0.154151,0.223144,0.287682,0.405465
firma_engen,59097.0,0.240711,0.632251,0.000000e+00,0.000000,0.000000,0.000000,5.568345
firma_share,59097.0,0.001115,0.004967,5.847944e-09,0.000011,0.000062,0.000378,0.252814
firma_potot_idade,59097.0,1.203585,0.589484,1.954530e-03,0.783232,1.156933,1.551576,7.214130
firma_potot_massa,59097.0,0.296870,0.187812,2.096026e-03,0.212821,0.291238,0.376063,6.600389


In [21]:
%%time
spread.filter(regex='firma', axis=1).corr()

Wall time: 77.8 ms


,firma_enc,firma_potot,firma_idade,firma_massa,firma_po3gr,firma_rotat,firma_engen,firma_share,firma_potot_idade,firma_potot_massa,firma_potot_po3gr,firma_idade_massa,firma_idade_po3gr,firma_idade_engen,firma_massa_po3gr,firma_massa_engen,firma_po3gr_engen,firma_rotat_engen,firma_rotat_share,firma_engen_share
firma_enc,1.000000,-0.010853,0.024123,-0.018079,-0.109492,0.016024,-0.170148,-0.112893,-0.003662,0.034546,0.132270,0.034635,0.077500,0.167406,0.048350,0.191996,0.005869,0.061877,0.018711,-0.170011
firma_potot,-0.010853,1.000000,0.562861,0.962429,0.852892,-0.104613,0.579890,0.380866,0.791167,0.559746,-0.005352,-0.059488,-0.564839,0.024164,-0.643270,0.115699,0.696688,-0.333911,-0.113546,0.579442
firma_idade,0.024123,0.562861,1.000000,0.567162,0.520785,-0.286245,0.312902,0.205638,0.020861,0.323991,-0.046446,0.289896,0.059414,0.640809,-0.407415,0.111229,0.457472,-0.385745,-0.290551,0.312660
firma_massa,-0.018079,0.962429,0.567162,1.000000,0.851508,-0.107659,0.568105,0.361730,0.758583,0.398438,-0.027943,-0.226835,-0.568748,0.040786,-0.614001,0.174741,0.707626,-0.331419,-0.116127,0.567711
firma_po3gr,-0.109492,0.852892,0.520785,0.851508,1.000000,-0.110849,0.645216,0.403040,0.617129,0.422906,-0.465635,-0.052864,-0.734252,-0.067946,-0.872751,-0.061085,0.831387,-0.359455,-0.120308,0.644799
firma_rotat,0.016024,-0.104613,-0.286245,-0.107659,-0.110849,1.000000,-0.015660,-0.080389,0.061900,-0.113353,0.013113,-0.158867,-0.089936,-0.240035,0.069102,-0.060350,-0.110475,0.897125,0.999703,-0.015369
firma_engen,-0.170148,0.579890,0.312902,0.568105,0.645216,-0.015660,1.000000,0.461923,0.413714,0.269926,-0.244399,-0.038798,-0.392859,-0.487368,-0.458596,-0.670774,0.147216,-0.405183,-0.026781,0.999994
firma_share,-0.112893,0.380866,0.205638,0.361730,0.403040,-0.080389,0.461923,1.000000,0.260255,0.176139,-0.127509,-0.020851,-0.224943,-0.157795,-0.263560,-0.211591,0.172613,-0.200610,-0.104640,0.458746
firma_potot_idade,-0.003662,0.791167,0.020861,0.758583,0.617129,0.061900,0.413714,0.260255,1.000000,0.455328,0.104352,-0.234812,-0.630278,-0.309826,-0.454173,0.144899,0.516893,-0.127345,0.055498,0.413440
firma_potot_massa,0.034546,0.559746,0.323991,0.398438,0.422906,-0.113353,0.269926,0.176139,0.455328,1.000000,0.129283,0.597021,-0.250163,0.069659,-0.397954,0.002864,0.362607,-0.220299,-0.117336,0.269722


In [22]:
%%time
spread.filter(regex='oper', axis=1).corr()

Wall time: 35.9 ms


,oper_valor,oper_spreadbndes,oper_juros,oper_amortizacao,oper_carencia,oper_valor_spreadbndes,oper_valor_juros,oper_valor_amortizacao,oper_valor_carencia,oper_spreadbndes_juros,oper_spreadbndes_amortizacao,oper_spreadbndes_carencia,oper_juros_amortizacao,oper_juros_carencia,oper_amortizacao_carencia
oper_valor,1.000000,0.001894,-0.082998,-0.017999,0.119088,0.963103,0.442593,0.077965,0.171839,0.087102,-0.006925,-0.124685,-0.021884,-0.143618,-0.028519
oper_spreadbndes,0.001894,1.000000,0.439267,-0.280726,0.078473,-0.267308,-0.394568,0.287715,-0.045006,-0.340261,0.310018,0.037314,0.341931,0.212503,-0.299802
oper_juros,-0.082998,0.439267,1.000000,-0.468580,-0.037531,-0.198199,-0.930363,0.450964,0.066276,-0.994218,0.466535,0.128063,0.600958,0.635669,-0.430714
oper_amortizacao,-0.017999,-0.280726,-0.468580,1.000000,0.098544,0.058209,0.415019,-0.946958,-0.102671,0.456919,-0.948220,-0.121443,-0.941217,-0.340641,0.914893
oper_carencia,0.119088,0.078473,-0.037531,0.098544,1.000000,0.093635,0.077585,-0.098277,-0.879206,0.048665,-0.106100,-0.914776,-0.104875,-0.735148,-0.153872
oper_valor_spreadbndes,0.963103,-0.267308,-0.198199,0.058209,0.093635,1.000000,0.532680,-0.002305,0.177699,0.175508,-0.090110,-0.130190,-0.113113,-0.195584,0.053206
oper_valor_juros,0.442593,-0.394568,-0.930363,0.415019,0.077585,0.532680,1.000000,-0.377105,0.003584,0.926670,-0.422349,-0.161107,-0.548810,-0.624831,0.377076
oper_valor_amortizacao,0.077965,0.287715,0.450964,-0.946958,-0.098277,-0.002305,-0.377105,1.000000,0.133850,-0.437644,0.996137,0.135570,0.981189,0.341188,-0.958854
oper_valor_carencia,0.171839,-0.045006,0.066276,-0.102671,-0.879206,0.177699,0.003584,0.133850,1.000000,-0.074752,0.117701,0.952575,0.120064,0.778515,0.153003
oper_spreadbndes_juros,0.087102,-0.340261,-0.994218,0.456919,0.048665,0.175508,0.926670,-0.437644,-0.074752,1.000000,-0.451277,-0.129586,-0.588166,-0.639969,0.415003


<a name="partition"></a>
    
## 1.4) Train/Test partition 

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [23]:
%%time
%%time
files = spread.columns
selected_files = files.str.contains('firma|oper|banco|seg|enc')
features = files[selected_files]
lista = list(features)
lista.remove('Numero_Agencias')
lista.remove('prazo_carencia')
lista.sort()

Wall time: 970 µs
Wall time: 1.96 ms


In [24]:
lista

['banco_ativo',
 'banco_ativo_captacoes',
 'banco_ativo_carteira',
 'banco_ativo_passivo',
 'banco_ativo_patrimonio',
 'banco_captacoes',
 'banco_captacoes_patrimonio',
 'banco_carteira',
 'banco_carteira_captacoes',
 'banco_carteira_passivo',
 'banco_carteira_patrimonio',
 'banco_enc',
 'banco_lucroliquido',
 'banco_numagencias',
 'banco_numeropostos',
 'banco_passivo',
 'banco_passivo_captacoes',
 'banco_passivo_patrimonio',
 'banco_patrimonio',
 'firma_enc',
 'firma_engen',
 'firma_engen_share',
 'firma_idade',
 'firma_idade_engen',
 'firma_idade_massa',
 'firma_idade_po3gr',
 'firma_massa',
 'firma_massa_engen',
 'firma_massa_po3gr',
 'firma_po3gr',
 'firma_po3gr_engen',
 'firma_potot',
 'firma_potot_idade',
 'firma_potot_massa',
 'firma_potot_po3gr',
 'firma_rotat',
 'firma_rotat_engen',
 'firma_rotat_share',
 'firma_share',
 'oper_amortizacao',
 'oper_amortizacao_carencia',
 'oper_carencia',
 'oper_juros',
 'oper_juros_amortizacao',
 'oper_juros_carencia',
 'oper_spreadbndes',
 '

In [25]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

X, y = spread[lista] , spread[['target']]

Xmin_max = MinMaxScaler().fit_transform(X)
ymin_max = MinMaxScaler().fit_transform(y)

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X , y , test_size=0.30)

X_train0, X_test0, y_train0, y_test0 = train_test_split(Xmin_max , ymin_max , test_size=0.30)

<a name="randomsearch"></a>

# 2) Random Search for Hyperparameters

- [Extreme Learning Machine](#elm)
- [Suport Vector Machine](#svm)
- [Stochastic Gradient Descent](#sgd)
- [Multi-Task Elastic-Net](#elastic)
- [Least Angle Regression Model (LAR)](#LAR)
- [Ridge Regression](#ridge)
- [ElasticNet Regression](#ElasNet)
- [Bayesian Ridge](#bayesian)
- [Lasso Regression](#lasso)
- [Multilayer Perceptron Regression](#mlp)
- [Nearest Neighbors Regression](#nnr)
- [Resumo dos resultados](#resumo_rs)

<a name="elm"></a>

## 2.1) Extreme learning machine

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [6]:
%%time
from elmx import ELMClassifier, ELMRegressor, GenELMClassifier, GenELMRegressor
from random_layer import RandomLayer, MLPRandomLayer, RBFRandomLayer, GRBFRandomLayer
from sklearn.svm import SVC
from sklearn.experimental import enable_halving_search_cv 
#from sklearn.model_selection import HalvingRandomSearchCV, HalvingGridSearchC


import sklearn
print(sklearn.__version__)

0.24.2
Wall time: 1 ms


In [30]:
%%time
from elmx import ELMClassifier, ELMRegressor, GenELMClassifier, GenELMRegressor
from random_layer import RandomLayer, MLPRandomLayer, RBFRandomLayer, GRBFRandomLayer
from sklearn.svm import SVC

space = dict()
space['hidden_layer__activation_func'] = ['tanh','sine','tribas','sigmoid','hardlim','softlim','gaussian','multiquadric','inv_multiquadric']
space['hidden_layer__n_hidden']        = [row for row in range(2,20,2)]
space['hidden_layer__rbf_width']       = [row/80 for row in range(1,80,5)]
space['hidden_layer__activation_args'] = [{'power':1.5}, {'power':2.5}, {'power':3.0}, {'power':3.5},
                                          {'power':4.0}, {'power':4.5}, {'power':5.5}, {'power':6.5}]

elm = GenELMRegressor(hidden_layer= RBFRandomLayer() )

elm0= RandomizedSearchCV(elm, space ,  cv=3, random_state=1, n_iter=300)
#elm0 = HalvingRandomSearchCV(svm, space, n_candidates = 500 , max_resources=50, 
#                             random_state=666, cv=3, factor=3, n_jobs=20 , scoring='f1')

elm0.fit(X_train, y_train)

Wall time: 1min 6s


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=GenELMRegressor(hidden_layer=RBFRandomLayer(activation_args=None,
                                                                         activation_func='gaussian',
                                                                         centers=None,
                                                                         n_hidden=20,
                                                                         radii=None,
                                                                         random_state=None,
                                                                         rbf_width=1.0),
                                             regressor=None),
                   iid='warn', n_iter=300, n_jobs=None,
                   param_distributions={'hidden_layer__activation_args': [{'power': 1.5},
                                                                          {'pow...
                

In [31]:
# summarize result
print('Best Score: %s' % elm0.best_score_) #Random Search Best Score: 0.9530819785561222
print('Best Hyperparameters: %s' % elm0.best_params_)

Best Score: 0.9904007039802292
Best Hyperparameters: {'hidden_layer__rbf_width': 0.0125, 'hidden_layer__n_hidden': 18, 'hidden_layer__activation_func': 'multiquadric', 'hidden_layer__activation_args': {'power': 4.0}}


In [32]:
%%time
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test,elm0.predict(X_test)))
print("EVS", explained_variance_score( y_test,elm0.predict(X_test)))
print("MAE", mean_absolute_error(      y_test,elm0.predict(X_test)))

R2: 0.9877717026607485
EVS 0.9877717255345593
MAE 0.25533507242057535
Wall time: 82.3 ms


<a name="svm"></a>

## 2.2) Suport Vector Machine


- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [33]:
%%time
from sklearn.svm import SVR

# define search space
space = dict()
space['kernel']    = ['linear','poly','rbf','sigmoid']
space['gamma']     = ['scale','auto']
space['coef0']     = [row/50 for row in range(1,100,4)]
space['C']         = [row/50 for row in range(1,100,4)]
space['epsilon']   = [row/25 for row in range(5,25)]
space['degree']    = [row/500 for row in range(1,250,5)]
space['tol']       = [row/2000 for row in range(1,200,10)]
space['shrinking'] = [True, False] 

svm0= RandomizedSearchCV(SVR(), space ,  cv=3, random_state=1, n_iter=75)

svm0.fit(X_train0, y_train0.ravel())

Wall time: 15min 12s


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                                 epsilon=0.1, gamma='auto_deprecated',
                                 kernel='rbf', max_iter=-1, shrinking=True,
                                 tol=0.001, verbose=False),
                   iid='warn', n_iter=75, n_jobs=None,
                   param_distributions={'C': [0.02, 0.1, 0.18, 0.26, 0.34, 0.42,
                                              0.5, 0.58, 0.66, 0.74, 0.82, 0.9,
                                              0.98, 1.06, 1.14, 1.22, 1.3...
                                                    0.8, 0.84, 0.88, 0.92,
                                                    0.96],
                                        'gamma': ['scale', 'auto'],
                                        'kernel': ['linear', 'poly', 'rbf',
                                                   'sigmoid'],
                        

In [34]:
# summarize result
print('Best Score: %s' % svm0.best_score_) #Best Score: 0.9352753839517703
print('Best Hyperparameters: %s' % svm0.best_params_)

Best Score: 0.6125368009168928
Best Hyperparameters: {'tol': 0.0205, 'shrinking': False, 'kernel': 'linear', 'gamma': 'scale', 'epsilon': 0.2, 'degree': 0.452, 'coef0': 0.5, 'C': 1.7}


In [35]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0 ,svm0.predict(X_test0)))
print("EVS", explained_variance_score( y_test0 ,svm0.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0 ,svm0.predict(X_test0)))

R2: 0.6124562342956525
EVS 0.861155976080434
MAE 0.08866761310337117


<a name="sgd"></a>

## 2.3) Stochastic Gradient Descent 

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [60]:
%%time
from sklearn.linear_model import SGDRegressor

# define search space
space = dict()
space['alpha']          = [row/50 for row in range(40,500,25)]
space['l1_ratio']       = [row/400 for row in range(20,100,4)] 
space['loss']           = ['squared_loss','huber','epsilon_insensitive','squared_epsilon_insensitive']
space['penalty']        = ['l1','l2','elasticnet']
space['learning_rate']  = ['invscaling','constant','optimal','adaptive']
space['epsilon']        = [row/700 for row in range(10,200,20)]
space['eta0']           = [row/1200 for row in range(1,200,10)]
space['power_t']        = [row/600 for row in range(60,300,10)]
space['early_stopping'] = [False,True]
space['tol']            = [row/5000 for row in range(1,60,5)]

sgd = SGDRegressor(max_iter=2000)

sgd0 = RandomizedSearchCV(sgd, space, cv=3, random_state=666, n_iter=300)

sgd0.fit(X_train0, y_train0.ravel())

Wall time: 3min 48s


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=SGDRegressor(alpha=0.0001, average=False,
                                          early_stopping=False, epsilon=0.1,
                                          eta0=0.01, fit_intercept=True,
                                          l1_ratio=0.15,
                                          learning_rate='invscaling',
                                          loss='squared_loss', max_iter=2000,
                                          n_iter_no_change=5, penalty='l2',
                                          power_t=0.25, random_state=None,
                                          shuffle=True, tol=0.001,
                                          validation_fraction=0...
                                                    0.31666666666666665,
                                                    0.3333333333333333, 0.35,
                                                    0.36666666666666664,
       

In [61]:
# summarize result
print('Best Score: %s' % sgd0.best_score_) 
print('Best Hyperparameters: %s' % sgd0.best_params_)

Best Score: 0.24783362368048545
Best Hyperparameters: {'tol': 0.0012, 'power_t': 0.48333333333333334, 'penalty': 'l2', 'loss': 'epsilon_insensitive', 'learning_rate': 'adaptive', 'l1_ratio': 0.2, 'eta0': 0.12583333333333332, 'epsilon': 0.014285714285714285, 'early_stopping': False, 'alpha': 3.8}


In [62]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0.ravel(),sgd0.predict(X_test0)))
print("EVS", explained_variance_score( y_test0.ravel(),sgd0.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0.ravel(),sgd0.predict(X_test0)))

R2: 0.2519237011592609
EVS 0.3177865752298399
MAE 0.0905236023795528


<a name="elastic"></a>

## 2.4) Multi-Task Elastic-Net

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [63]:
%%time
from sklearn import linear_model

space = dict()
space['alpha']         = [row/200 for row in range(1,1200,60)]
space['l1_ratio']      = [row/30 for row in range(1,300,10)]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['warm_start']    = [False,True]
space['copy_X']        = [False,True]
space['selection']     = ['cyclic','random']
space['tol']           = [row/5000 for row in range(1,20,2)]

mtask = linear_model.MultiTaskElasticNet(max_iter=800)

mtask0 = RandomizedSearchCV(mtask, space, cv=3,  n_iter=300, random_state=1)

mtask0.fit(X_train, y_train)
#X_train0, X_test0, y_train0, y_test0 = train_test_split(Xmin_max , ymin_max , test_size=0.30)

Wall time: 12min 45s


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=MultiTaskElasticNet(alpha=1.0, copy_X=True,
                                                 fit_intercept=True,
                                                 l1_ratio=0.5, max_iter=800,
                                                 normalize=False,
                                                 random_state=None,
                                                 selection='cyclic', tol=0.0001,
                                                 warm_start=False),
                   iid='warn', n_iter=300, n_jobs=None,
                   param_distributions={'alpha': [0.005, 0.305, 0.605, 0.905,
                                                  1.205, 1.505, 1.805, 2.105,
                                                  2.4...
                                                     7.366666666666666, 7.7,
                                                     8.033333333333333,
                      

In [64]:
# summarize result
print('Best Score: %s' % mtask0.best_score_)
print('Best Hyperparameters: %s' % mtask0.best_params_)

Best Score: 0.991052351294834
Best Hyperparameters: {'warm_start': True, 'tol': 0.0006, 'selection': 'cyclic', 'normalize': False, 'l1_ratio': 1.0333333333333334, 'fit_intercept': False, 'copy_X': False, 'alpha': 0.005}


In [65]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test,mtask0.predict(X_test)))
print("EVS", explained_variance_score( y_test,mtask0.predict(X_test)))
print("MAE", mean_absolute_error(      y_test,mtask0.predict(X_test)))

R2: 0.9907340718253873
EVS 0.9907340788900089
MAE 0.22493605198946276


<a name="LAR"></a>

## 2.5) Least Angle Regression Model (LAR)

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [66]:
%%time
from sklearn import linear_model

space = dict()
space['alpha']          = [row/300 for row in range(1,900,10)]
space['fit_intercept']  = [False,True]
space['fit_path']       = [False,True]
space['normalize']      = [False,True]
space['copy_X']         = [False,True]
space['positive']       = [False,True]
space['eps']            = [row/100 for row in range(1000,10000,100)]

lars = linear_model.LassoLars()

lars0 = RandomizedSearchCV(lars, space, cv=3,  n_iter=75, random_state=666)

lars0.fit(X_train, y_train)

Wall time: 10.2 s


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=LassoLars(alpha=1.0, copy_X=True,
                                       eps=2.220446049250313e-16,
                                       fit_intercept=True, fit_path=True,
                                       max_iter=500, normalize=True,
                                       positive=False, precompute='auto',
                                       verbose=False),
                   iid='warn', n_iter=75, n_jobs=None,
                   param_distributions={'alpha': [0.0033333333333333335,
                                                  0.03666666666666667, 0.07,
                                                  0.10333...
                                        'eps': [10.0, 11.0, 12.0, 13.0, 14.0,
                                                15.0, 16.0, 17.0, 18.0, 19.0,
                                                20.0, 21.0, 22.0, 23.0, 24.0,
                                         

In [67]:
# summarize result
print('Best Score: %s' % lars0.best_score_) #Best Score: 0.9471256547797681
print('Best Hyperparameters: %s' % lars0.best_params_)

Best Score: 0.990968527258223
Best Hyperparameters: {'positive': False, 'normalize': True, 'fit_path': False, 'fit_intercept': False, 'eps': 22.0, 'copy_X': False, 'alpha': 0.03666666666666667}


In [68]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test,lars0.predict(X_test)))
print("EVS", explained_variance_score( y_test,lars0.predict(X_test)))
print("MAE", mean_absolute_error(      y_test,lars0.predict(X_test)))

R2: 0.9906394974188831
EVS 0.9906396949717318
MAE 0.2293304026831187


<a name="ridge"></a>

## 2.6) Ridge Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [69]:
%%time
from sklearn.linear_model import Ridge
   
space = dict()
space['alpha']         = [row/500 for row in range(8,100,5)]
space['copy_X']        = [False,True]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['solver']        = ['auto','svd','cholesky','lsqr','sparse_cg']
space['tol']           = [row/100000 for row in range(1,1000,10)]

ridge = Ridge()

ridge0 = RandomizedSearchCV(ridge, space, cv=3,  n_iter=300 , random_state=666)

ridge0.fit(X_train, y_train)

Wall time: 55 s
Parser   : 138 ms


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                                   max_iter=None, normalize=False,
                                   random_state=None, solver='auto',
                                   tol=0.001),
                   iid='warn', n_iter=300, n_jobs=None,
                   param_distributions={'alpha': [0.016, 0.026, 0.036, 0.046,
                                                  0.056, 0.066, 0.076, 0.086,
                                                  0.096, 0.106, 0.116, 0.126,
                                                  0.136, 0.146, 0.156, 0.166,
                                                  0.17...
                                        'tol': [1e-05, 0.00011, 0.00021,
                                                0.00031, 0.00041, 0.00051,
                                                0.00061, 0.00071, 0.00081,
                               

In [70]:
# summarize result
print('Best Score: %s' % ridge0.best_score_) #Best Score: 0.9581611723130965
print('Best Hyperparameters: %s' % ridge0.best_params_)

Best Score: 0.9918170169364112
Best Hyperparameters: {'tol': 0.00151, 'solver': 'cholesky', 'normalize': True, 'fit_intercept': False, 'copy_X': False, 'alpha': 0.016}


In [71]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test, ridge0.predict(X_test)))
print("EVS", explained_variance_score( y_test, ridge0.predict(X_test)))
print("MAE", mean_absolute_error(      y_test, ridge0.predict(X_test)))

R2: 0.9915792067560228
EVS 0.9915792300898715
MAE 0.21841804855791583


<a name="ElasNet"></a>

## 2.7) ElasticNet Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [72]:
%%time
from sklearn.linear_model import ElasticNet

space = dict()
space['alpha']         = [row/500 for row in range(1,500,5)]
space['l1_ratio']      = [row/250 for row in range(5,1000,10)]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['precompute']    = [False,True]
space['copy_X']        = [False,True]
space['warm_start']    = [False,True]
space['positive']      = [False,True]
space['selection']     = ['cyclic','random']
space['tol']           = [row/10000 for row in range(10,10000,10)]

elastic = ElasticNet()

elastic0 = RandomizedSearchCV(elastic, space, cv=3,  n_iter=300, random_state=666)

elastic0.fit(X_train0, y_train0)

Wall time: 42.1 s


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=ElasticNet(alpha=1.0, copy_X=True,
                                        fit_intercept=True, l1_ratio=0.5,
                                        max_iter=1000, normalize=False,
                                        positive=False, precompute=False,
                                        random_state=None, selection='cyclic',
                                        tol=0.0001, warm_start=False),
                   iid='warn', n_iter=300, n_jobs=None,
                   param_distributions={'alpha': [0.002, 0.012, 0.022, 0.032,
                                                  0.0...
                                        'precompute': [False, True],
                                        'selection': ['cyclic', 'random'],
                                        'tol': [0.001, 0.002, 0.003, 0.004,
                                                0.005, 0.006, 0.007, 0.008,
                     

In [73]:
# summarize result
print('Best Score: %s' % elastic0.best_score_) #Best Score: 0.902707307723327
print('Best Hyperparameters: %s' % elastic0.best_params_)

Best Score: 0.9683606923477642
Best Hyperparameters: {'warm_start': False, 'tol': 0.43, 'selection': 'random', 'precompute': True, 'positive': False, 'normalize': False, 'l1_ratio': 1.58, 'fit_intercept': True, 'copy_X': False, 'alpha': 0.002}


In [74]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, elastic0.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, elastic0.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, elastic0.predict(X_test0)))

R2: 0.9231841849747465
EVS 0.9231845259415319
MAE 0.03299100298768805


<a name="bayesian"></a>

## 2.8) Bayesian Ridge Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [51]:
%%time
from sklearn.linear_model import BayesianRidge

space = dict()
space['alpha_1']        = [row/40000000 for row in range(1,2000,50)]
space['alpha_2']        = [row/40000000 for row in range(1,2000,50)]
space['lambda_1']       = [row/40000000 for row in range(1,2000,50)]
space['lambda_2']       = [row/40000000 for row in range(1,2000,50)]
space['compute_score']  = [False,True]
space['copy_X']         = [False,True]
space['fit_intercept']  = [False,True]
space['normalize']      = [False,True]
space['tol']            = [row/5000 for row in range(1,100,5)]

bayes = BayesianRidge(n_iter=800)

bayes0 = RandomizedSearchCV(bayes, space, cv=3,  n_iter=75 , random_state=666)

bayes0.fit(X_train0, y_train0.ravel())

Wall time: 35.2 s


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=BayesianRidge(alpha_1=1e-06, alpha_2=1e-06,
                                           compute_score=False, copy_X=True,
                                           fit_intercept=True, lambda_1=1e-06,
                                           lambda_2=1e-06, n_iter=800,
                                           normalize=False, tol=0.001,
                                           verbose=False),
                   iid='warn', n_iter=75, n_jobs=None,
                   param_distributions={'alpha_1': [2.5e-08, 1.275e-06,
                                                    2.525e-06, 3.775e-06,
                                                    5.025e...
                                                     2.7525e-05, 2.8775e-05,
                                                     3.0025e-05, 3.1275e-05,
                                                     3.2525e-05, 3.3775e-05,
                  

In [51]:
# summarize result
print('Best Score: %s' % bayes0.best_score_)
print('Best Hyperparameters: %s' % bayes0.best_params_)

Best Score: 0.9544850768185378
Best Hyperparameters: {'tol': 0.0192, 'normalize': False, 'lambda_2': 6.275e-06, 'lambda_1': 4.1275e-05, 'fit_intercept': False, 'copy_X': False, 'compute_score': True, 'alpha_2': 4.5025e-05, 'alpha_1': 1.275e-06}


In [52]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, bayes0.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, bayes0.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, bayes0.predict(X_test0)))

R2: 0.9590290461883912
EVS 0.9591084576233943
MAE 0.017742315785688287


<a name="lasso"></a>

## 2.9) Lasso Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [52]:
%%time
from sklearn.linear_model import Lasso

space = dict()
space['alpha']         = [row/1000 for row in range(10,100,5)]
space['copy_X']        = [False,True]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['precompute']    = [False,True]
space['positive']      = [False,True]
space['selection']     = ['cyclic','random']
space['tol']           = [row/1000000 for row in range(100,10000,10)]

lasso = Lasso()

lasso0 = RandomizedSearchCV(lasso, space, cv=3, n_iter=75, random_state=666)

lasso0.fit(X_train0, y_train0)

Wall time: 17.4 s


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                                   max_iter=1000, normalize=False,
                                   positive=False, precompute=False,
                                   random_state=None, selection='cyclic',
                                   tol=0.0001, warm_start=False),
                   iid='warn', n_iter=75, n_jobs=None,
                   param_distributions={'alpha': [0.01, 0.015, 0.02, 0.025,
                                                  0.03, 0.035, 0.04, 0.045,
                                                  0....
                                        'tol': [0.0001, 0.00011, 0.00012,
                                                0.00013, 0.00014, 0.00015,
                                                0.00016, 0.00017, 0.00018,
                                                0.00019, 0.0002, 0.00021,
                          

In [53]:
# summarize result
print('Best Score: %s' % lasso0.best_score_) #Best Score: 0.8776039138486295
print('Best Hyperparameters: %s' % lasso0.best_params_) 

Best Score: 0.9035396626705005
Best Hyperparameters: {'tol': 0.00233, 'selection': 'cyclic', 'precompute': True, 'positive': False, 'normalize': False, 'fit_intercept': False, 'copy_X': True, 'alpha': 0.01}


In [54]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, lasso0.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, lasso0.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, lasso0.predict(X_test0)))

R2: 0.903587970465622
EVS 0.9071484976304887
MAE 0.036009073318471196



<a name="mlp"></a>

## 2.10) Multilayer Perceptron

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [75]:
%%time
from sklearn.neural_network import MLPRegressor

space = dict()
space['hidden_layer_sizes']  = [(100,50), (200,25), (25,100,25), (50,50,50), (75,50,75), (80,60,80)]
space['activation']          = ['identity','logistic','tanh','relu']
space['solver']              = ['lbfgs', 'sgd', 'adam']
space['alpha']               = [row/200 for row in range(1,100,10)]
space['learning_rate']       = ['constant','invscaling','adaptive']
space['learning_rate_init']  = [row/1000 for row in range(1,20,8)]
space['warm_start']          = [False,True]
space['power_t']             = [row/50 for row in range(1,200,20)]
space['tol']                 = [row/50000000 for row in range(1,1000,200)]
space['momentum']            = [row/200 for row in range(1,100,10)]
space['nesterovs_momentum']  = [False,True]
space['early_stopping']      = [False,True]
space['beta_1']              = [row/200 for row in range(1,20,6)]
space['beta_2']              = [row/200 for row in range(1,20,6)]
space['epsilon']             = [row/200 for row in range(1,20,6)]
    
mpl = MLPRegressor(max_iter=2000)

mpl0 = RandomizedSearchCV(mpl, space, cv=3,  n_iter=150, random_state=666)

mpl0.fit(X_train0, y_train0.ravel())

Wall time: 1d 21h 15min 10s


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=MLPRegressor(activation='relu', alpha=0.0001,
                                          batch_size='auto', beta_1=0.9,
                                          beta_2=0.999, early_stopping=False,
                                          epsilon=1e-08,
                                          hidden_layer_sizes=(100,),
                                          learning_rate='constant',
                                          learning_rate_init=0.001,
                                          max_iter=2000, momentum=0.9,
                                          n_iter_no_change=10,
                                          nesterovs_momentum=True, power_t=0.5,
                                          rand...
                                        'momentum': [0.005, 0.055, 0.105, 0.155,
                                                     0.205, 0.255, 0.305, 0.355,
                            

In [76]:
# summarize result
print('Best Score: %s' % mpl0.best_score_) #Best Score: 0.9550165534562787
print('Best Hyperparameters: %s' % mpl0.best_params_)

Best Score: 0.9934342418902639
Best Hyperparameters: {'warm_start': True, 'tol': 8.02e-06, 'solver': 'lbfgs', 'power_t': 2.02, 'nesterovs_momentum': True, 'momentum': 0.055, 'learning_rate_init': 0.001, 'learning_rate': 'constant', 'hidden_layer_sizes': (200, 25), 'epsilon': 0.035, 'early_stopping': False, 'beta_2': 0.065, 'beta_1': 0.005, 'alpha': 0.005, 'activation': 'relu'}


In [77]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, mpl0.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, mpl0.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, mpl0.predict(X_test0)))

R2: 0.9932139931313203
EVS 0.9932149339463296
MAE 0.010078692422783772


<a name="nnr"></a>

## 2.11) Nearest Neighbors Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [ ]:
%%time
from sklearn.neighbors import KNeighborsRegressor

space = dict()
space['n_neighbors'] = [row for row in range(4,10)]
space['weights']     = ['uniform', 'distance']
space['algorithm']   = ['auto', 'ball_tree', 'kd_tree', 'brute']
space['leaf_size']   = [row for row in range(10,50)]
space['p']           = [row for row in range(2,8)]
    
knn = KNeighborsRegressor()

knn0 = RandomizedSearchCV(knn, space, cv=3, random_state=666, n_iter=75)

knn0.fit(X_train0, y_train0)

In [ ]:
# summarize result
print('Best Score: %s' % knn0.best_score_)
print('Best Hyperparameters: %s' % knn0.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, knn0.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, knn0.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, knn0.predict(X_test0)))

<a name="resumo_rs"></a>

## 2.12) Resumo dos resultados

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [89]:
R2_mpl      = r2_score(                 y_test0, mpl0.predict(X_test0))
EVS_mpl     = explained_variance_score( y_test0, mpl0.predict(X_test0))
MAE_mpl     = mean_absolute_error(      y_test0, mpl0.predict(X_test0))

R2_lasso    = r2_score(                 y_test0, lasso0.predict(X_test0))
EVS_lasso   = explained_variance_score( y_test0, lasso0.predict(X_test0))
MAE_lasso   = mean_absolute_error(      y_test0, lasso0.predict(X_test0))

R2_bayes    = r2_score(                 y_test0, bayes0.predict(X_test0))
EVS_bayes   = explained_variance_score( y_test0, bayes0.predict(X_test0))
MAE_bayes   = mean_absolute_error(      y_test0, bayes0.predict(X_test0))

R2_elastic  = r2_score(                 y_test0, elastic0.predict(X_test0))
EVS_elastic = explained_variance_score( y_test0, elastic0.predict(X_test0))
MAE_elastic = mean_absolute_error(      y_test0, elastic0.predict(X_test0))

R2_ridge    = r2_score(                 y_test , ridge0.predict(X_test))
EVS_ridge   = explained_variance_score( y_test , ridge0.predict(X_test))
MAE_ridge   = mean_absolute_error(      y_test , ridge0.predict(X_test))

R2_lars     = r2_score(                 y_test ,lars0.predict(X_test))
EVS_lars    = explained_variance_score( y_test ,lars0.predict(X_test))
MAE_lars    = mean_absolute_error(      y_test ,lars0.predict(X_test))

R2_sgd      = r2_score(                 y_test ,sgd0.predict(X_test))
EVS_sgd     = explained_variance_score( y_test ,sgd0.predict(X_test))
MAE_sgd     = mean_absolute_error(      y_test ,sgd0.predict(X_test))

R2_svm      = r2_score(                 y_test ,svm0.predict(X_test))
EVS_svm     = explained_variance_score( y_test ,svm0.predict(X_test))
MAE_svm     = mean_absolute_error(      y_test ,svm0.predict(X_test))

R2_elm      = r2_score(                 y_test ,elm0.predict(X_test))
EVS_elm     = explained_variance_score( y_test ,elm0.predict(X_test))
MAE_elm     = mean_absolute_error(      y_test ,elm0.predict(X_test))

R2_knn      = r2_score(                 y_test0, knn0.predict(X_test0))
EVS_knn     = explained_variance_score( y_test0, knn0.predict(X_test0))
MAE_knn     = mean_absolute_error(      y_test0, knn0.predict(X_test0))


In [90]:
RSQ = [R2_mpl , R2_lasso , R2_bayes , R2_elastic   , R2_ridge  , R2_lars  , R2_sgd , R2_svm , R2_elm , R2_knn]
EVS = [EVS_mpl, EVS_lasso, EVS_bayes, EVS_elastic  , EVS_ridge , EVS_lars , EVS_sgd, EVS_svm, EVS_elm, EVS_knn]
MAE = [MAE_mpl, MAE_lasso, MAE_bayes, MAE_elastic  , MAE_ridge , MAE_lars , MAE_sgd, MAE_svm, MAE_elm, MAE_knn]
MOD = ['MLP'  , 'Lasso'  , 'Bayes'  ,'Elastic Net' , 'Ridge'   , 'LARS'   , 'SGD'  ,'SVM Reg', 'ELM' , 'KNN']
BHY = [ mpl0.best_params_ ,lasso0.best_params_  ,bayes0.best_params_  ,elastic0.best_params_ ,ridge0.best_params_ ,
        lars0.best_params_ ,sgd0.best_params_  ,svm0.best_params_ ,elm0.best_params_  ,knn0.best_params_]

In [91]:
resumo_rs = {'Mod' : MOD, 
             'R2'  : RSQ, 
             'EVS' : EVS, 
             'MAE' : MAE,
             'BHY' : BHY}

In [92]:
pd_resumo_rs = pd.DataFrame(data=resumo_rs)
pd_resumo_rs

,Mod,R2,EVS,MAE,BHY
0,MLP,0.955204,0.955288,0.017714,"{'warm_start': True, 'tol': 8.02e-06, 'solver'..."
1,Lasso,0.908398,0.956472,0.035478,"{'tol': 0.00233, 'selection': 'cyclic', 'preco..."
2,Bayes,0.959029,0.959108,0.017742,"{'tol': 0.0192, 'normalize': False, 'lambda_2'..."
3,Elastic Net,0.885522,0.954300,0.041032,"{'warm_start': False, 'tol': 0.131, 'selection..."
4,Ridge,0.953250,0.953273,0.086062,"{'tol': 0.00681, 'solver': 'cholesky', 'normal..."
5,LARS,0.948160,0.948177,0.090254,"{'positive': False, 'normalize': True, 'fit_pa..."
6,SGD,0.896835,0.896841,0.145870,"{'tol': 0.0012, 'power_t': 0.48333333333333334..."
7,SVM Reg,-494.500742,0.075403,15.077866,"{'tol': 0.028, 'shrinking': True, 'kernel': 's..."
8,ELM,0.952405,0.952419,0.086448,"{'hidden_layer__rbf_width': 0.0125, 'hidden_la..."
9,KNN,0.955125,0.955190,0.019260,"{'weights': 'distance', 'p': 3, 'n_neighbors':..."


In [66]:
pd_resumo_rs.to_csv('resumo_rs.csv')

<a name="gridsearch"></a>

# Grid Serach Hyperparameter Tunning


- [Extreme Learning Machine](#elm2)
- [Suport Vector Machine](#svm2)
- [Stochastic Gradient Descent](#sgd2)
- [Multi-Task Elastic-Net](#elastic2)
- [Least Angle Regression Model (LAR)](#LAR2)
- [Ridge Regression](#ridge2)
- [ElasticNet Regression](#ElasNet2)
- [Bayesian Ridge](#bayesian2)
- [Lasso Regression](#lasso2)
- [Multilayer Perceptron Regression](#mlp2)
- [Nearest Neighbors Regression](#nnr2)


<a name="elm2"></a>

## 3.1) Extreme learning machine

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [2]:
0.1375-0.0125

0.125

In [35]:
%%time
from elmx import ELMClassifier, ELMRegressor, GenELMClassifier, GenELMRegressor
from random_layer import RandomLayer, MLPRandomLayer, RBFRandomLayer, GRBFRandomLayer

space = dict()
space['hidden_layer__activation_func'] = ['tanh','sine','tribas','sigmoid','hardlim','softlim','gaussian','multiquadric','inv_multiquadric']
space['hidden_layer__n_hidden']        = [row for row in range(2,20,4)]
space['hidden_layer__rbf_width']       = [row/80 for row in range(1,80,10)]
space['hidden_layer__activation_args'] = [{'power':1.5}, {'power':2.5}, {'power':3.0}, {'power':3.5},
                                          {'power':4.0}, {'power':4.5}, {'power':5.5}, {'power':6.5}]

elm = GenELMRegressor(hidden_layer= RBFRandomLayer() )

elm1= GridSearchCV(elm, space, cv=3)

elm1.fit(X_train, y_train)

Wall time: 1min 53s


GridSearchCV(cv=3, estimator=GenELMRegressor(hidden_layer=RBFRandomLayer()),
             param_grid={'hidden_layer__activation_args': [{'power': 1.5},
                                                           {'power': 2.5},
                                                           {'power': 3.0},
                                                           {'power': 3.5},
                                                           {'power': 4.0},
                                                           {'power': 4.5},
                                                           {'power': 5.5},
                                                           {'power': 6.5}],
                         'hidden_layer__activation_func': ['tanh', 'sine',
                                                           'tribas', 'sigmoid',
                                                           'hardlim', 'softlim',
                                                           'gaussian',
               

In [36]:
# summarize result
print('Best Score: %s' % elm1.best_score_) #Best Score: 0.0.9501519111182998
print('Best Hyperparameters: %s' % elm1.best_params_)

Best Score: 0.9554508542438648
Best Hyperparameters: {'hidden_layer__activation_args': {'power': 4.0}, 'hidden_layer__activation_func': 'inv_multiquadric', 'hidden_layer__n_hidden': 14, 'hidden_layer__rbf_width': 0.0125}


In [37]:
%%time
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test,elm1.predict(X_test)))
print("EVS", explained_variance_score( y_test,elm1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test,elm1.predict(X_test)))

R2: 0.9609808590880473
EVS 0.9609811717093774
MAE 0.08183364801158495
Wall time: 18.4 ms


<a name="svm2"></a>

## 3.2) Suport Vector Machine

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [47]:
%%time
from sklearn.svm import SVR
    
# define search space
space = dict()
space['kernel']    = ['rbf']
#space['kernel']    = ['linear','poly','rbf','sigmoid']
space['gamma']     = ['scale']
space['coef0']     = [row/50 for row in range(1,100,8)]
space['C']         = [row/50 for row in range(1,100,8)]
space['epsilon']   = [row/25 for row in range(5,25,4)]
space['degree']    = [row/500 for row in range(1,250,10)]
space['tol']       = [row/2000 for row in range(1,200,20)]
space['shrinking'] = [True] 

svm1= GridSearchCV(SVR(), space,  cv=3)

svm1.fit(X_train0, y_train0.ravel())

Wall time: 37min 2s


GridSearchCV(cv=3, estimator=SVR(),
             param_grid={'C': [0.02, 0.18, 0.34, 0.5, 0.66, 0.82, 0.98, 1.14,
                               1.3, 1.46, 1.62, 1.78, 1.94],
                         'coef0': [0.02, 0.18, 0.34, 0.5, 0.66, 0.82, 0.98,
                                   1.14, 1.3, 1.46, 1.62, 1.78, 1.94],
                         'degree': [0.002, 0.022, 0.042, 0.062, 0.082, 0.102,
                                    0.122, 0.142, 0.162, 0.182, 0.202, 0.222,
                                    0.242, 0.262, 0.282, 0.302, 0.322, 0.342,
                                    0.362, 0.382, 0.402, 0.422, 0.442, 0.462,
                                    0.482],
                         'epsilon': [0.2, 0.36, 0.52, 0.68, 0.84],
                         'gamma': ['scale'], 'kernel': ['rbf'],
                         'shrinking': [True],
                         'tol': [0.0005, 0.0105, 0.0205, 0.0305, 0.0405, 0.0505,
                                 0.0605, 0.0705, 0.0805, 0.0905]

In [48]:
# summarize result
print('Best Score: %s' % svm1.best_score_)
print('Best Hyperparameters: %s' % svm1.best_params_)

Best Score: 0.6649117450062915
Best Hyperparameters: {'C': 0.5, 'coef0': 0.02, 'degree': 0.002, 'epsilon': 0.2, 'gamma': 'scale', 'kernel': 'rbf', 'shrinking': True, 'tol': 0.0405}


In [49]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, svm1.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, svm1.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, svm1.predict(X_test0)))

R2: 0.770933849003383
EVS 0.8288177144557141
MAE 0.05568750525449417


<a name="sgd2"></a>

## 3.3) Stochastic Gradient Descent 

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [ ]:
%%time
from sklearn.linear_model import SGDRegressor

# define search space
space = dict()
space['alpha']          = [row/50 for row in range(40,500,10)]
space['l1_ratio']       = [row/400 for row in range(20,100,8)] 
space['loss']           = ['squared_loss']
space['penalty']        = ['l2']
space['learning_rate']  = ['invscaling']
space['epsilon']        = [row/700 for row in range(10,200,10)]
space['eta0']           = [row/1200 for row in range(1,200,10)]
space['power_t']        = [row/600 for row in range(60,300,10)]
space['early_stopping'] = [False]
space['tol']            = [row/5000 for row in range(1,60,20)]

sgd = SGDRegressor(max_iter=1500)

sgd1 = GridSearchCV(sgd, space, cv=3)

sgd1.fit(X_train0, y_train0.ravel())

In [ ]:
# summarize result
print('Best Score: %s' % sgd1.best_score_)
print('Best Hyperparameters: %s' % sgd1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, sgd1.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, sgd1.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, sgd1.predict(X_test0)))

<a name="elastic2"></a>

## 3.4) Multi-Task Elastic-Net

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [102]:
%%time
from sklearn import linear_model

space = dict()
space['alpha']         = [row/200 for row in range(1,1200,120)]
space['l1_ratio']      = [row/30 for row in range(1,300,20)]
space['fit_intercept'] = [True]
space['normalize']     = [False]
space['warm_start']    = [False]
space['copy_X']        = [True]
space['selection']     = ['cyclic','random']
space['tol']           = [row/5000 for row in range(1,20,4)]

mtask = linear_model.MultiTaskElasticNet(max_iter=800)

mtask1 = GridSearchCV(mtask, space, cv=3)

mtask1.fit(X_train, y_train)

Wall time: 44.3 s


GridSearchCV(cv=3, estimator=MultiTaskElasticNet(max_iter=800),
             param_grid={'alpha': [0.005, 0.605, 1.205, 1.805, 2.405, 3.005,
                                   3.605, 4.205, 4.805, 5.405],
                         'copy_X': [True], 'fit_intercept': [True],
                         'l1_ratio': [0.03333333333333333, 0.7,
                                      1.3666666666666667, 2.033333333333333,
                                      2.7, 3.3666666666666667,
                                      4.033333333333333, 4.7, 5.366666666666666,
                                      6.033333333333333, 6.7, 7.366666666666666,
                                      8.033333333333333, 8.7,
                                      9.366666666666667],
                         'normalize': [False],
                         'selection': ['cyclic', 'random'],
                         'tol': [0.0002, 0.001, 0.0018, 0.0026, 0.0034],
                         'warm_start': [False]})

In [103]:
# summarize result
print('Best Score: %s' % mtask1.best_score_)
print('Best Hyperparameters: %s' % mtask1.best_params_)

Best Score: 0.9568539998858528
Best Hyperparameters: {'alpha': 0.005, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.03333333333333333, 'normalize': False, 'selection': 'cyclic', 'tol': 0.0002, 'warm_start': False}


In [104]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test,mtask1.predict(X_test)))
print("EVS", explained_variance_score( y_test,mtask1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test,mtask1.predict(X_test)))

R2: -200.3819292706111
EVS 0.952090224751859
MAE 9.620150701925867


<a name="LAR2"></a>

## 3.5) Least Angle Regression Model (LAR)

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [59]:
%%time
from sklearn import linear_model

space = dict()
space['alpha']          = [row/300 for row in range(1,900,20)]
space['fit_intercept']  = [True]
space['fit_path']       = [True]
space['normalize']      = [True]
space['copy_X']         = [True]
space['positive']       = [False]
space['eps']            = [row/100 for row in range(1000,10000,100)]

lars = linear_model.LassoLars()

lars1 = GridSearchCV(lars, space, cv=3)

lars1.fit(X_train, y_train)

Wall time: 3min 25s


GridSearchCV(cv=3, estimator=LassoLars(),
             param_grid={'alpha': [0.0033333333333333335, 0.07,
                                   0.13666666666666666, 0.20333333333333334,
                                   0.27, 0.33666666666666667,
                                   0.4033333333333333, 0.47, 0.5366666666666666,
                                   0.6033333333333334, 0.67, 0.7366666666666667,
                                   0.8033333333333333, 0.87, 0.9366666666666666,
                                   1.0033333333333334, 1.07, 1.1366666666666667,
                                   1.2033333333333334, 1.27, 1...
                                   1.4033333333333333, 1.47, 1.5366666666666666,
                                   1.6033333333333333, 1.67, 1.7366666666666666,
                                   1.8033333333333332, 1.87, 1.9366666666666668, ...],
                         'copy_X': [True],
                         'eps': [10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0

In [60]:
# summarize result
print('Best Score: %s' % lars1.best_score_)
print('Best Hyperparameters: %s' % lars1.best_params_)

Best Score: 0.13618103863881806
Best Hyperparameters: {'alpha': 0.0033333333333333335, 'copy_X': True, 'eps': 10.0, 'fit_intercept': True, 'fit_path': True, 'normalize': True, 'positive': False}


In [61]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test,lars1.predict(X_test)))
print("EVS", explained_variance_score( y_test,lars1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test,lars1.predict(X_test)))

R2: 0.12340541855729359
EVS 0.12341317784041284
MAE 0.52586417452343


<a name="ridge2"></a>

## 3.6) Ridge Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [56]:
%%time
from sklearn.linear_model import Ridge

space = dict()
space['alpha']         = [row/500 for row in range(8,100,5)]
space['copy_X']        = [True]
space['fit_intercept'] = [True]
space['normalize']     = [False]
space['solver']        = ['auto','svd','cholesky','lsqr','sparse_cg']
space['tol']           = [row/100000 for row in range(1,1000,10)]

ridge = Ridge()

ridge1 = GridSearchCV(ridge, space, cv=3)

ridge1.fit(X_train, y_train)

Wall time: 7min 44s


GridSearchCV(cv=3, estimator=Ridge(),
             param_grid={'alpha': [0.016, 0.026, 0.036, 0.046, 0.056, 0.066,
                                   0.076, 0.086, 0.096, 0.106, 0.116, 0.126,
                                   0.136, 0.146, 0.156, 0.166, 0.176, 0.186,
                                   0.196],
                         'copy_X': [True], 'fit_intercept': [True],
                         'normalize': [False],
                         'solver': ['auto', 'svd', 'cholesky', 'lsqr',
                                    'sparse_cg'],
                         'tol': [1e-05, 0.00011, 0.00021, 0.00031, 0.00041,
                                 0.00051, 0.00061, 0.00071, 0.00081, 0.00091,
                                 0.00101, 0.00111, 0.00121, 0.00131, 0.00141,
                                 0.00151, 0.00161, 0.00171, 0.00181, 0.00191,
                                 0.00201, 0.00211, 0.00221, 0.00231, 0.00241,
                                 0.00251, 0.00261, 0.00271, 0.00

In [57]:
# summarize result
print('Best Score: %s' % ridge1.best_score_)
print('Best Hyperparameters: %s' % ridge1.best_params_)

Best Score: 0.9555557360485869
Best Hyperparameters: {'alpha': 0.186, 'copy_X': True, 'fit_intercept': True, 'normalize': False, 'solver': 'svd', 'tol': 1e-05}


In [58]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test, ridge1.predict(X_test)))
print("EVS", explained_variance_score( y_test, ridge1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test, ridge1.predict(X_test)))

R2: 0.9601717103695852
EVS 0.9601992664656231
MAE 0.08264653075280715


<a name="ElasNet2"></a>

## 3.7) ElasticNet Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [ ]:
%%time
from sklearn.linear_model import ElasticNet

space = dict()
space['alpha']         = [row/500 for row in range(1,500,10)]
space['l1_ratio']      = [row/250 for row in range(5,1000,20)]
space['fit_intercept'] = [True]
space['normalize']     = [False]
space['precompute']    = [False]
space['copy_X']        = [True]
space['warm_start']    = [False]
space['positive']      = [False]
space['selection']     = ['cyclic','random']
space['tol']           = [row/10000 for row in range(10,10000,20)]

elastic = ElasticNet()

elastic1 = GridSearchCV(elastic, space, cv=3)

elastic1.fit(X_train, y_train)

In [ ]:
# summarize result
print('Best Score: %s' % elastic0.best_score_)
print('Best Hyperparameters: %s' % elastic0.best_params_)

In [ ]:
# summarize result
print('Best Score: %s' % elastic0.best_score_)
print('Best Hyperparameters: %s' % elastic0.best_params_)

<a name="bayesian2"></a>

## 3.8) Bayesian Ridge Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [62]:
%%time   
from sklearn.linear_model import BayesianRidge

space = dict()
space['alpha_1']        = [row/40000000 for row in range(1,2000,100)]
space['alpha_2']        = [row/40000000 for row in range(1,2000,100)]
space['lambda_1']       = [row/40000000 for row in range(1,2000,100)]
space['lambda_2']       = [row/40000000 for row in range(1,2000,100)]
space['compute_score']  = [False]
space['copy_X']         = [True]
space['fit_intercept']  = [True]
space['normalize']      = [False]
space['tol']            = [row/5000 for row in range(1,100,10)]

bayes = BayesianRidge()

bayes1 = GridSearchCV(bayes, space, cv=3)

bayes1.fit(X_train, y_train.values.ravel())

Wall time: 1d 1h 14min 36s


GridSearchCV(cv=3, estimator=BayesianRidge(),
             param_grid={'alpha_1': [2.5e-08, 2.525e-06, 5.025e-06, 7.525e-06,
                                     1.0025e-05, 1.2525e-05, 1.5025e-05,
                                     1.7525e-05, 2.0025e-05, 2.2525e-05,
                                     2.5025e-05, 2.7525e-05, 3.0025e-05,
                                     3.2525e-05, 3.5025e-05, 3.7525e-05,
                                     4.0025e-05, 4.2525e-05, 4.5025e-05,
                                     4.7525e-05],
                         'alpha_2': [2.5e-08, 2.525e-06, 5.025e-06, 7.525e-06,
                                     1.0025e-05, 1.2525e-...
                                      4.0025e-05, 4.2525e-05, 4.5025e-05,
                                      4.7525e-05],
                         'lambda_2': [2.5e-08, 2.525e-06, 5.025e-06, 7.525e-06,
                                      1.0025e-05, 1.2525e-05, 1.5025e-05,
                                      1.7

In [63]:
# summarize result
print('Best Score: %s' % bayes1.best_score_)
print('Best Hyperparameters: %s' % bayes1.best_params_)

Best Score: 0.9553132641296713
Best Hyperparameters: {'alpha_1': 2.5e-08, 'alpha_2': 4.7525e-05, 'compute_score': False, 'copy_X': True, 'fit_intercept': True, 'lambda_1': 4.7525e-05, 'lambda_2': 4.7525e-05, 'normalize': False, 'tol': 0.0002}


In [64]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test, bayes1.predict(X_test)))
print("EVS", explained_variance_score( y_test, bayes1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test, bayes1.predict(X_test)))

R2: 0.9602185288479533
EVS 0.9602484101074821
MAE 0.08261676986087549


<a name="lasso2"></a>

## 3.9) Lasso Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [50]:
%%time
from sklearn.linear_model import Lasso

space = dict()
space['alpha']         = [row/1000 for row in range(10,100,10)]
space['copy_X']        = [True]
space['fit_intercept'] = [True]
space['normalize']     = [False]
space['precompute']    = [False]
space['positive']      = [False]
space['selection']     = ['cyclic','random']
space['tol']           = [row/1000000 for row in range(100,10000,20)]

lasso = Lasso()

lasso1 = GridSearchCV(lasso, space, cv=3)

lasso1.fit(X_train, y_train.values.ravel())

Wall time: 4min 44s


GridSearchCV(cv=3, estimator=Lasso(),
             param_grid={'alpha': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07,
                                   0.08, 0.09],
                         'copy_X': [True], 'fit_intercept': [True],
                         'normalize': [False], 'positive': [False],
                         'precompute': [False],
                         'selection': ['cyclic', 'random'],
                         'tol': [0.0001, 0.00012, 0.00014, 0.00016, 0.00018,
                                 0.0002, 0.00022, 0.00024, 0.00026, 0.00028,
                                 0.0003, 0.00032, 0.00034, 0.00036, 0.00038,
                                 0.0004, 0.00042, 0.00044, 0.00046, 0.00048,
                                 0.0005, 0.00052, 0.00054, 0.00056, 0.00058,
                                 0.0006, 0.00062, 0.00064, 0.00066, 0.00068, ...]})

In [51]:
# summarize result
print('Best Score: %s' % lasso1.best_score_)
print('Best Hyperparameters: %s' % lasso1.best_params_)

Best Score: 0.948382878285614
Best Hyperparameters: {'alpha': 0.01, 'copy_X': True, 'fit_intercept': True, 'normalize': False, 'positive': False, 'precompute': False, 'selection': 'random', 'tol': 0.00658}


In [52]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test, lasso1.predict(X_test)))
print("EVS", explained_variance_score( y_test, lasso1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test, lasso1.predict(X_test)))

R2: 0.9540382662526427
EVS 0.9540442153995186
MAE 0.09152546452262968


<a name="mlp2"></a>

## 3.10) Multilayer Perceptron

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [ ]:
%%time
from sklearn.neural_network import MLPRegressor

space = dict()
space['hidden_layer_sizes']  = [(100,50), (25,100,25), (75,50,75)]
space['activation']          = ['identity','logistic','tanh','relu']
space['solver']              = ['lbfgs', 'sgd', 'adam']
space['alpha']               = [row/200 for row in range(1,100,20)]
space['learning_rate']       = ['constant','invscaling','adaptive']
space['learning_rate_init']  = [row/1000 for row in range(1,20,10)]
space['warm_start']          = [False]
space['power_t']             = [row/50 for row in range(1,200,40)]
space['tol']                 = [row/50000000 for row in range(1,1000,400)]
space['momentum']            = [row/200 for row in range(1,100,20)]
space['nesterovs_momentum']  = [True]
space['early_stopping']      = [False]
space['beta_1']              = [row/200 for row in range(1,20,12)]
space['beta_2']              = [row/200 for row in range(1,20,12)]
space['epsilon']             = [row/200 for row in range(1,20,12)]
    
mpl = MLPRegressor(max_iter=1200)

mpl0 = GridSearchCV(mpl, space, cv=3 )

mpl0.fit(X_train0, y_train0.ravel())

In [ ]:
# summarize result
print('Best Score: %s' % mpl1.best_score_)
print('Best Hyperparameters: %s' % mpl1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, mpl1.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, mpl1.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, mpl1.predict(X_test0)))

<a name="nnr2"></a>

## 3.11) Nearest Neighbors Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [ ]:
%%time
from sklearn.neighbors import KNeighborsRegressor

space = dict()
space['n_neighbors'] = [row for row in range(4,10,2)]
space['weights']     = ['uniform', 'distance']
space['algorithm']   = ['auto', 'ball_tree', 'kd_tree', 'brute']
space['leaf_size']   = [row for row in range(10,50,2)]
space['p']           = [row for row in range(2,8,2)]
    
knn = KNeighborsRegressor()

knn1 = GridSearchCV(knn, space, cv=3)

knn1.fit(X_train, y_train)

In [ ]:
# summarize result
print('Best Score: %s' % knn1.best_score_)
print('Best Hyperparameters: %s' % knn1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test, knn1.predict(X_test)))
print("EVS", explained_variance_score( y_test, knn1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test, knn1.predict(X_test)))

<a name="resumo_rs2"></a>

## 3.12) Resumo dos resultados

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Grid Search Tunning](#gridsearch)

In [ ]:
R2_mpl      = r2_score(                 y_test0, mpl1.predict(X_test0))
EVS_mpl     = explained_variance_score( y_test0, mpl1.predict(X_test0))
MAE_mpl     = mean_absolute_error(      y_test0, mpl1.predict(X_test0))

R2_lasso    = r2_score(                 y_test0, lasso1.predict(X_test0))
EVS_lasso   = explained_variance_score( y_test0, lasso1.predict(X_test0))
MAE_lasso   = mean_absolute_error(      y_test0, lasso1.predict(X_test0))

R2_bayes    = r2_score(                 y_test0, bayes1.predict(X_test0))
EVS_bayes   = explained_variance_score( y_test0, bayes1.predict(X_test0))
MAE_bayes   = mean_absolute_error(      y_test0, bayes1.predict(X_test0))

R2_elastic  = r2_score(                 y_test0, elastic1.predict(X_test0))
EVS_elastic = explained_variance_score( y_test0, elastic1.predict(X_test0))
MAE_elastic = mean_absolute_error(      y_test0, elastic1.predict(X_test0))

R2_ridge    = r2_score(                 y_test , ridge1.predict(X_test))
EVS_ridge   = explained_variance_score( y_test , ridge1.predict(X_test))
MAE_ridge   = mean_absolute_error(      y_test , ridge1.predict(X_test))

R2_lars     = r2_score(                 y_test ,lars1.predict(X_test))
EVS_lars    = explained_variance_score( y_test ,lars1.predict(X_test))
MAE_lars    = mean_absolute_error(      y_test ,lars1.predict(X_test))

R2_sgd      = r2_score(                 y_test ,sgd1.predict(X_test))
EVS_sgd     = explained_variance_score( y_test ,sgd1.predict(X_test))
MAE_sgd     = mean_absolute_error(      y_test ,sgd1.predict(X_test))

R2_svm      = r2_score(                 y_test0 ,svm1.predict(X_test0))
EVS_svm     = explained_variance_score( y_test0 ,svm1.predict(X_test0))
MAE_svm     = mean_absolute_error(      y_test0 ,svm1.predict(X_test0))

R2_elm      = r2_score(                 y_test ,elm1.predict(X_test))
EVS_elm     = explained_variance_score( y_test ,elm1.predict(X_test))
MAE_elm     = mean_absolute_error(      y_test ,elm1.predict(X_test))

R2_knn      = r2_score(                 y_test0, knn1.predict(X_test0))
EVS_knn     = explained_variance_score( y_test0, knn1.predict(X_test0))
MAE_knn     = mean_absolute_error(      y_test0, knn1.predict(X_test0))

In [ ]:
RSQ = [R2_mpl , R2_lasso , R2_bayes , R2_elastic   , R2_ridge  , R2_lars  , R2_sgd , R2_svm , R2_elm , R2_knn]
EVS = [EVS_mpl, EVS_lasso, EVS_bayes, EVS_elastic  , EVS_ridge , EVS_lars , EVS_sgd, EVS_svm, EVS_elm, EVS_knn]
MAE = [MAE_mpl, MAE_lasso, MAE_bayes, MAE_elastic  , MAE_ridge , MAE_lars , MAE_sgd, MAE_svm, MAE_elm, MAE_knn]
MOD = ['MLP'  , 'Lasso'  , 'Bayes'  ,'Elastic Net' , 'Ridge'   , 'LARS'   , 'SGD'  ,'SVM Reg', 'ELM' , 'KNN']
BHY = [ mpl1.best_params_ ,lasso1.best_params_  ,bayes1.best_params_  ,elastic1.best_params_ ,
        ridge1.best_params_ ,lars1.best_params_ ,sgd1.best_params_  ,svm1.best_params_ ,
        elm1.best_params_  ,knn1.best_params_]

In [ ]:
resumo_gs = {'Mod' : MOD, 
             'R2'  : RSQ, 
             'EVS' : EVS, 
             'MAE' : MAE,
             'BHY' : BHY}

In [ ]:
resumo_gs = pd.DataFrame(data=resumo_gs)
resumo_gs

In [ ]:
pd_resumo_gs.to_csv('resumo_gs.csv')